# Calling Julia from Python

Import the Python module that we will need later

In [1]:
import time as tm
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt

---

## Type inference examples ( _from earlier_ )

In [2]:
type(2 ** 3)

int

In [3]:
type(2 ** -3)

float

In [4]:
type(np.sqrt(1))

numpy.float64

In [5]:
type(np.sqrt(-1))

/Users/malcolm/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in sqrt
  if __name__ == '__main__':


numpy.float64

In [6]:
type(np.sqrt(-1 + 0j))

numpy.complex128

In [7]:
def fac(n):
 if n < 2:
   return 1
 else:
   return n*fac(n-1)

In [8]:
fac(20)

2432902008176640000

In [9]:
fac(21)

51090942171709440000L

### Bring in the Julia interpretor

In [10]:
from julia import Julia

In [11]:
jl = Julia()

In [12]:
jl.bessely0(1.5) * np.sin(1.5)

0.38149088412038057

In [13]:
x = [0.1*i for i in range(300)]

In [14]:
y = [jl.gamma(0.015*(i+1)) * np.sin(0.15*i)  for i in range(300)]

In [15]:
plt.plot(x,y)
plt.show()

---

### Series expansion for log(x), |x| < 1.0, is very slow as x -> 1.0

log(1+x) = x - x<sup>2</sup>/2 + x<sup>3</sup>/3 - x<sup>4</sup>/4 + x<sup>5</sup>/5 - . . .


In [ ]:
def slogp(x,n):
   if (n > 0 and abs(x) < 1):
      s = 0.0
      for i in range(n):
         j = i + 1
         s += ((-1)**i) * (x**j / float(j))
      return s
   else:
      raise ValueError('Illegal parameter values')

In [ ]:
slogp(0.99995,1000)

In [ ]:
t0 = tm.time(); slogp(0.99995,10000000); print (tm.time() - t0)

In [ ]:
slogj = jl.eval("""
function slog(x::Real,n::Integer)
  @assert abs(n) > 0
  @assert abs(x) < 1.0
  s = 0.0
  for i in 1:n
    s += (-1)^(i+1) * (x^i / i)
  end
  return s
end
""")

In [ ]:
slogj(0.99995,1000)

In [ ]:
slogj(0.99995,10000000)

In [ ]:
t0 = tm.time(); slogj(0.99995,10000000); print (tm.time() - t0)

---

### Vander function is difficult to compute as a vectorised process
<p>
A Vandermonde matrix, named after Alexandre-Théophile Vandermonde, is a matrix with the terms of a geometric progression in each row, i.e., an m × n matrix.</p>
<p>
1&nbsp;&nbsp;a&nbsp;&nbsp;a<sup>2</sup>&nbsp;a<sup>3</sup> . . . . . . . . . . . . . . . .&nbsp;a<sup>(n-1)</sup><br/>
1&nbsp;&nbsp;b&nbsp;&nbsp;b<sup>2</sup>&nbsp;b<sup>3</sup> . . . . . . . . . . . . . . . .&nbsp;b<sup>(n-1)</sup><br/>
1&nbsp;&nbsp;c&nbsp;&nbsp;c<sup>2</sup>&nbsp;c<sup>3</sup> . . . . . . . . . . . . . . . .&nbsp;c<sup>(n-1)</sup><br/>
. . .<br/>
. . .<br/>
. . .<br/>
1&nbsp;&nbsp;m&nbsp;&nbsp;m<sup>2</sup>&nbsp;m<sup>3</sup> . . . . . . . . . . . . . . . .&nbsp;m<sup>(n-1)</sup><br/>
</p>
<p>
Vandermonde matrices are used in linear algebra (Hermite interpolation), DFT (discrete Fourier transforms) and Group theory.</p>
<p>
They are also used in some forms of BCH and Reed–Solomon error correction codes.</p>
<p>
These are an important group of error-correcting codes which have many important applications, which include technologies such as CDs, DVDs, Blu-ray Discs, QR Codes, data transmission technologies such as DSL and WiMAX, broadcast systems such as DVB and ATSC, and storage systems such as RAID 6; they are also used in satellite communication.</p>


The PYTHON (numpy) code is quite straight forward

```python
def vander(x,N):
  x = np.asarray(x)
  if x.ndim != 1:
    raise ValueError("x must be a 1-D array or sequence")
  v = np.empty((len(x), N), dtype=np.promote_types(x.dtype, int))
  if N > 0:
    v[:,0] = 1
  if N > 1:
    v[:, 1:] = x[:, None]
    np.multiply.accumulate(v[:, 1:], out=v[:, 1:], axis=1)
  return v

```

But the accumulate function is very complex, written in C and is 347 lines long.

In [ ]:
x = nr.rand(1000); 

In [ ]:
np.vander(x,50)

The Julia version is much closer to algorithm

```
function vander(x, N::Int)
  x = convert(AbstractVector, x)
  M = length(x)
  v = Array(promote_type(eltype(x),Int), M, N)
  if N > 0 
    v[:, 1] = 1
  end
  if N > 1
    for i = 2:N
      v[:,i] = x
    end
    accumulate(*,v,v)
  end
  return v
end

function accumulate(op, input, output)
  M, N = size(input)
  for i = 2:N
    for j = 1:M
      output[j,i] = op(input[j,i], input[j,i-1])
    end
  end
end

```

The accumulate function is written in Julia and only 6 lines of code

In [ ]:
jl.eval('pwd()')

In [ ]:
jl.call('include("./code/vander.jl")')

In [ ]:
jl.eval("xx = rand(1000);")
jl.eval("@timed vander(xx,50)")

---

# Using the Cosmology module

In [8]:
jl.eval('using Cosmology')

In [9]:
jl.eval('csm = cosmology(OmegaM=0.26, OmegaK=0.1, OmegaR=0.11, Tcmb=3.1)')

<PyCall.jlwrap OpenLCDM(0.69,0.1,0.53,0.26,0.11)>

In [10]:
jl.eval('angular_diameter_dist_mpc(csm, 1.3)')

1562.4172706124527

In [11]:
jl.eval('age_gyr(csm, 1.3)')

3.0292101139640337

In [12]:
jl.eval('lookback_time_gyr(csm, 1.3)')

7.950649088454039

---

# Asian option price
( Compare with the Python code: _asianOpt.py_ )

In [13]:
jl.eval("""
function asianOpt(N=1000, T=100; S0=100.0, K=100.0, r=0.05, q=0.0, v=0.2, tma=0.25	) 

# European Asian option.  
# Euler and Milstein discretization for Black-Scholes.

  dt = tma/T;      # Time increment

  S = zeros(Float64,T);
  A = zeros(Float64,N);

# Main calculation loop

  for n = 1:N
    S[1] = S0 
    dW = randn(T)*sqrt(dt);
    for t = 2:T
      z0 = (r - q - 0.5*v*v)*S[t-1]*dt;
      z1 = v*S[t-1]*dW[t];
      z2 = 0.5*v*v*S[t-1]*dW[t]*dW[t];
      S[t] = S[t-1] + z0 + z1 + z2;
    end
    A[n] = mean(S);
  end

# Define the payoff and calculate price

  P = zeros(Float64,N);
  [ P[n] = max(A[n] - K, 0) for n = 1:N ];
  price = exp(-r*tma)*mean(P);

end
""")

<PyCall.jlwrap asianOpt>

In [14]:
rts = jl.eval('@timed asianOpt(1000000,100; K=102.0)')
rts

(1.676897861724939, 3.245925463, 1823036520, 1.061756212)

In [15]:
print "Option price is ", rts[0]
print "Time taken was  ", rts[1], "sec."

Option price is  1.67689786172
Time taken was   3.245925463 sec.
